In [20]:
# Import of the packages
import numpy as np
import scipy.optimize as optimize
import matplotlib.pyplot as plt
import math
import scipy.stats as sts

In [21]:
class ParameterValueError(Exception):
    """ Custom exception raised when a parameter has an invalid value."""
    pass

In [332]:
# RVM class
class RVM:
    """ Relevance Vector Machine (RVM)
    
    Implementation of RVM for both regression and classification.
    
    Attributes:
        kerType: A string of the type of the desired kernel.
        rvmType: A string denoting the RVM type to be used. The string "EM" denotes
            an EM-like algorithm will be used to estimate the hyperparameters while
            the string "DD" denotes the direct differentiation approach.
        p: Integer value denoting the degree of the polynomial kernel.
        sigma: Float value denoting the smoothing factor of the Gaussian kernel.
        kappa: Float value denoting the scaling parameter of the sigmoid kernel.
        delta: Float value denoting the translation parameter of the sigmoid kernel.
        bTrained: boolean value which becomes true once the RVM has been trained.
    """

    EPSILON = 1e-10
    maxEpochs = 10000
    
    def __init__(self, kerType = 'poly', rvmType = "EM", p = 1, sigma = 1, 
                 kappa = 1, delta = 1):
        """ Initializes the RVM class (constructor).
        
            Raises:
                ParameterValueError: An error occured because a parameter had an
                    invalid value.  
        """
        # Check if the kernel type chosen is valid
        kerTypes = ['linear', 'poly', 'radial', 'sigmoid']
        if kerType not in kerTypes:
            raise ParameterValueError("ParameterValueError: The string " + kerType + \
                                       " does not denote a valid kernel type")
        # Check if the string denoting the rvmType has a valid value
        if rvmType != 'EM' and rvmType != 'DD':
            raise ParameterValueError('ParameterValueError: ' + rvmType, \
                                       " is not a valid RVM type value. Enter 'EM' or 'DD' as a value. ")
       
        self.kerType = kerType
        self.rvmType = rvmType
        self.p = p
        self.sigma = sigma
        self.kappa = kappa
        self.delta = delta
        self.bTrained = False

    def kernel(self, x, y):
        """ Kernel computation.
        
        It computes the kernel value based on the dot product
        between two vectors.
        
        Args:
            x: Input vector.
            y: Other input vector.
            
        Returns:
            The computed kernel value.
        """  
        if self.kerType == "linear":
            k = np.dot(x,y) + 1
        elif self.kerType == "poly":
            k = (np.dot(x,y) + 1) ** self.p
        elif self.kerType == "radial":
            k = math.exp(-(np.dot(x-y,x-y))/(2*self.sigma))
        elif self.kerType == "sigmoid":
            k = math.atanh(self.kappa * np.dot(x,y) - self.delta)

        return k
    
    def getKernelMatrix(self, X):
        """ Evaluates the kernel matrix K given a set of input samples.

            Args:
                X_tr: A NxM matrix with a M dimensional training input sample
                    in each row.
                
            Returns:
                An NxN Kernel matrix where N is the number of input samples.
        """
        N = X.shape[0]
        K = np.zeros((N, N))
        for i in range(N):
            for j in range(N):
                K[i,j] = self.kernel(X[i], X[j])
        return K

    def getGammaValues(self, alpha_values, Sigma):
        """Evaluates the gamma values.
        
            Args:
                alpha_values: N-dimensional vector with the hyperparameters of
                    the marginal likelihood.
                Sigma: NxN covariance matrix of the posterior
                
            Returns: A N-dimensional vector with the gamma values where 
                gamma_values[i] = 1 - alpha_values[i] * Sigma[i][i]
        """
        N = alpha_values.shape[0]
        gamma_values = np.zeros(N)
        for i in range(N):
            gamma_values[i] = 1 - alpha_values[i] * Sigma[i][i]
        return gamma_values
        
    def train(self, X_tr, Y_tr):
        """ RVM training method
        
        Applies an EM-like algorithm or direct differentiation to estimate the
        optimal hyperparameters (alpha and sigma) needed to make predictions
        using the marginal likelihood.
        
        
        Args:
            X_tr: A matrix with a training input sample in each row.
            Y_tr: A vector with the output values of each input sample
                in X_tr.
        
        Returns:
            None
        """
        # Get number of training data samples
        N = X_tr.shape[0]
        # Initialize the alpha values (weight precision values) and the A matrix
        alpha_values = np.ones(N + 1)
        A = np.diag(alpha_values)
        # Initialize the sigma squared value and the B matrix
        sigma_squared = 1
        B = np.identity(N) * (sigma_squared ** -2)
        # Calculate kernel matrix K and append a column with ones in the front (for the bias term??)
        K = self.getKernelMatrix(X_tr)
        K = np.hstack((np.ones(N).reshape((N, 1)), K))
        # Calculate Sigma and mu based on the initialized parameters
        Sigma = np.linalg.inv(K.T.dot(B).dot(K) + A)
        mu = Sigma.dot(K.T).dot(B).dot(Y_tr)
        # Calculate initial gamma values
        gamma_values = self.getGammaValues(alpha_values, Sigma)
        
        

        # Approximate optimal hyperparameter values iteratively
        next_alpha_values = np.zeros(N + 1)
        for epoch in range(self.maxEpochs):
            # Evaluate alpha values
            for i in range(N + 1):
                if self.rvmType == "EM":
                    next_alpha_values[i] = 1 / (Sigma[i][i] + mu[i] ** 2)
                elif self.rvmType == "DD":
                    pass                
            # Evaluate sigma value
            next_sigma_squared = (np.linalg.norm(Y_tr - K.dot(mu)) ** 2) / (N - np.sum(gamma_values))
            # Check if algorithm has converged
            if (np.sum(np.absolute(next_alpha_values - alpha_values)) < self.EPSILON and
                (next_sigma_squared - sigma_squared) < self.EPSILON):
                    break
            # If algorithm has not converged, update all the variables
            alpha_values = next_alpha_values
            sigma_squared = next_sigma_squared
            Sigma = np.linalg.inv(K.T.dot(B).dot(K) + A)
            mu = Sigma.dot(K.T).dot(B).dot(Y_tr)
            gamma_values = self.getGammaValues(alpha_values, Sigma)
        

        

In [333]:
N_tr = 50
N_ts = 1000

def trans(x):
    N = np.shape(x)[0]
    y = np.sin(x) + 3 + np.random.normal(loc=0, scale=0.1, size=N)
    return y 

x_tr = np.linspace(0,7.5,num=N_tr)
y_tr = trans(x_tr)
x_ts = np.linspace(0,7.5,num=N_ts)
y_ts = trans(x_ts)

In [334]:
try:
    s = RVM()
    s.train(x_tr, y_tr)
except Exception as e:
    print(e)
    

0
1
[ 200.79180475  186.81550781  157.68442771  157.97751557  193.6186864
  177.45152228  234.7208384   202.68187488  206.20478011  225.67309225
  201.32685239  219.89547736  264.60644634  256.62431969  254.63050932
  240.74345339  348.47591518  327.40923444  331.36294078  383.63794812
  372.47254874  457.7569987   422.04580905  405.46582726  391.41849156
  443.3042352   492.97269601  523.25051542  446.10990332  522.4861964
  406.5092793   540.23384992  490.03569974  460.07158067  458.77340662
  533.09457963  523.7292509   487.06679078  523.6783194   482.77736217
  446.41348477  508.06217927  501.32744151  454.13332438  433.04518716
  442.13050021  540.21591208  386.64123155  447.48026452  369.88209667
  378.13106048]
